In [10]:
import numpy as np
import random
import tensorflow as tf
import tensorflow_datasets as tfds
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
import bert
import transformers as T
import re
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
import seaborn as sns
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import math

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train_text = open("C:/Users/HP/Desktop/AI/NLP/Emotions NLP/train.txt","r",encoding="utf8", errors="ignore").read().split("\n")
val_text = training = open("C:/Users/HP/Desktop/AI/NLP/Emotions NLP/val.txt","r",encoding="utf8", errors="ignore").read().split("\n")
test_text = training = open("C:/Users/HP/Desktop/AI/NLP/Emotions NLP/test.txt","r",encoding="utf8", errors="ignore").read().split("\n")

In [3]:
# Doing a first cleaning of the texts
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text) # replace "i'm" with "i am"
    text = re.sub(r"im", "i am", text)
    text = re.sub(r"ive", "i have", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"n't", "not", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"wont", "will not", text)
    text = re.sub(r"won t", "will not", text)
    text = re.sub(r"didn't", "did not", text)
    text = re.sub(r"didnt", "did not", text)
    text = re.sub(r"didn t", "did not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"cant", "cannot", text)
    text = re.sub(r"can t", "cannot", text)
    #text = re.sub(r"[-()\"#/@:<>{}+=~|.?,!]", "", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,!,...]", "", text)
    return text

In [4]:
def split(text):
    X = []
    y = []
    #new_text = text.split('\n')
    for i in text:
        if ";" not in i:
            pass
            #X.append(i)
        else:
            ali = i.split(";")
            X.append(clean_text(ali[0]))
            y.append(clean_text(ali[1]))
        
    return X, y

X_train, y_train_unmapped = split(train_text)
X_test, y_test_unmapped = split(test_text)
X_val, y_val_unmapped = split(val_text)

In [20]:
y = set(y_train_unmapped)
emotions_dict = {"love":0, "sadness":1, "anger":2, "surprise":3, "joy":4, "fear":5}

y_train = [str(emotions_dict[i]) for i in y_train_unmapped]
y_test = [str(emotions_dict[i]) for i in y_test_unmapped]
y_val = [str(emotions_dict[i]) for i in y_val_unmapped]

In [19]:
type(y_train[0])

int

In [8]:
tokenizer = T.T5Tokenizer.from_pretrained("t5-small")
model = T.T5ForConditionalGeneration.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [14]:
X_train_new = [torchtext.transforms.ToTensor(i) for i in X_train]
X_test_new = [torchtext.transforms.ToTensor(i) for i in X_test]
X_val_new = [torchtext.transforms.ToTensor(i) for i in X_val]

y_train_new = [torchtext.transforms.ToTensor(i) for i in y_train]
y_test_new = [torchtext.transforms.ToTensor(i) for i in y_test]
y_val_new = [torchtext.transforms.ToTensor(i) for i in y_val]

In [22]:
max_train_len = 68
max_test_len = 64
task_prefix = "classify sentiment: "

encoding = tokenizer(
    X_train,
    padding="longest",
    max_length=max_train_len,
    truncation=True,
    return_tensors="pt",
)

input_ids, attention_mask = encoding.input_ids, encoding.attention_mask


target_encoding = tokenizer(
    y_train,
    padding="longest",
    max_length=10,
    truncation=True,
    return_tensors="pt",
)

labels = target_encoding.input_ids

In [23]:
for i in range(5):
    loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
    print(loss.item())

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 2367488000 bytes.